In [9]:
import json
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

In [10]:
# Load the data
with open("intents.json") as file:
    intents = json.load(file)

In [11]:
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        # Add documents in the corpus
        documents.append((word_list, intent['tag']))
        # Add to our classes list if tag not already in classes
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [12]:
# Lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

print(f"Words: {words}")
print(f"Classes: {classes}")

Words: ["'m", "'re", "'s", 'a', 'aal', 'about', 'admit', 'all', 'am', 'any', 'apologize', 'apology', 'appreciate', 'are', 'at', 'away', 'awesome', 'bad', 'bedroom', 'believe', 'big', 'book', 'but', 'bye', 'can', 'catch', 'chesavu', 'cheyyuva', 'cuisine', 'current', 'cute', 'da', 'deal', 'dhanyavadh', 'dhanyavadhalu', 'di', 'do', 'doing', 'dream', 'durr', 'eh', 'elaga', 'em', 'enna', 'enth', 'entha', 'enti', 'epdi', 'epo', 'everything', 'excuse', 'fact', 'fault', 'favorite', 'fine', 'first', 'food', 'for', 'forecast', 'forgive', 'future', 'gadget', 'game', 'genre', 'go', 'going', 'good', 'goodbye', 'great', 'happen', 'have', 'hello', 'helloo', 'hey', 'hi', 'hii', 'hiii', 'hobby', 'how', 'i', 'ilana', 'in', 'iruka', 'irukangala', 'iruken', 'is', 'it', 'job', 'kind', 'kiss', 'ku', 'kya', 'lately', 'later', 'leaving', 'let', 'like', 'lot', 'love', 'mamey', 'me', 'mikka', 'miru', 'mistake', 'move', 'movie', 'music', 'mwahh', 'my', "n't", 'naa', 'naam', 'nalla', 'name', 'nandri', 'need', 'ni

In [13]:
# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    # Initialize bag of words
    bag = []
    # List of tokenized words for the pattern
    word_patterns = doc[0]
    # Lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # Create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in word_patterns else bag.append(0)
    
    # Output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Shuffle the features and convert it into a numpy array
np.random.shuffle(training)
training = np.array(training, dtype=object)  # Explicitly specify dtype=object

# Create train and test lists
train_x = np.array([np.array(element[0]) for element in training])
train_y = np.array([np.array(element[1]) for element in training])

print("Training data created")

Training data created


In [14]:
# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [15]:
# Compile the model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [16]:
# Train the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200)
model.save('chatbot_model.h5', hist)
print("Model created")

Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0694 - loss: 3.2633  
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0673 - loss: 3.2227 
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0609 - loss: 3.2531 
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0590 - loss: 3.2567 
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0652 - loss: 3.2292 
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1261 - loss: 3.1916 
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1052 - loss: 3.2282 
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0790 - loss: 3.2032 
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0842 - loss: 3.2014 
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0548 - loss: 3.1634 
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0948 - loss: 3.1473 
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0894 - lo

Model created
